In [ ]:
# Resume_Workflow - essential cells to run after restarting VS Code / kernel
# Kernel: use your `niva-notebook` kernel

# %%
# Cell 1 - load env, basic imports and connections
from dotenv import load_dotenv
import os, json
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX = os.getenv("PINECONE_INDEX","medical-knowledge")
OLLAMA_URL = os.getenv("OLLAMA_URL","http://127.0.0.1:11434")
MODEL_NAME = os.getenv("MODEL_NAME","llama3")
CSV_PATH = os.getenv("CSV_PATH","./data/medical_knowledge.csv")

print("PINECONE_API_KEY set?", bool(PINECONE_API_KEY))
print("OLLAMA_URL:", OLLAMA_URL)
print("CSV_PATH exists?", os.path.exists(CSV_PATH))

# Initialize Pinecone client (if you use Pinecone)
try:
    from pinecone import Pinecone
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index(PINECONE_INDEX)
    print("Connected to Pinecone index:", PINECONE_INDEX)
except Exception as e:
    print("Pinecone init failed (ok if not using now):", e)

# Load embedding model (SentenceTransformers)
try:
    from sentence_transformers import SentenceTransformer
    embed_model = SentenceTransformer("all-MiniLM-L6-v2")
    print("Embedding model ready. Dim:", len(embed_model.encode("test")))
except Exception as e:
    print("Embedding model load failed:", e)

# %%
# Cell 2 - robust call_ollama() that handles NDJSON streaming
import requests, re

essential_ollama_url = OLLAMA_URL.rstrip('/') + '/api/generate'

def call_ollama(prompt, max_tokens=500, temperature=0.0, timeout=60):
    payload = {"model": MODEL_NAME, "prompt": prompt, "max_tokens": max_tokens, "temperature": temperature, "stream": False}
    try:
        r = requests.post(essential_ollama_url, json=payload, timeout=timeout)
    except Exception as e:
        raise RuntimeError(f"Ollama request failed: {e}")

    text = (r.text or "").strip()
    # try single json
    try:
        data = r.json()
        if isinstance(data, dict):
            if "response" in data:
                return data["response"]
            if "output" in data:
                return data["output"]
    except Exception:
        pass

    # NDJSON fallback: parse line-by-line, concatenate 'response' fields
    out = ""
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            if isinstance(obj, dict) and obj.get("response"):
                out += str(obj.get("response"))
        except Exception:
            # ignore non-json lines
            continue
    return out.strip()

print("call_ollama ready")

# %%
# Cell 3 - next_question_rag() using embed_model + Pinecone index
from dotenv import load_dotenv
load_dotenv()

def next_question_rag(user_text, k=3):
    # require embed_model and index to be defined
    try:
        qvec = embed_model.encode(user_text).tolist()
    except Exception as e:
        raise RuntimeError(f"Embedding failed: {e}")

    try:
        res = index.query(vector=qvec, top_k=k, include_metadata=True)
        ctx_text = "\n\n".join([m['metadata'].get('text','') for m in res.get('matches',[])])
    except Exception as e:
        ctx_text = ""
        print("Pinecone query failed (context empty):", e)

    prompt = f"You are a concise medical triage assistant. Use ONLY the context below to ask ONE concise follow-up question that helps clarify the patient's symptom described.\n\nContext:\n{ctx_text}\n\nPatient statement:\n{user_text}\n\nAsk only the next question."
    return call_ollama(prompt, max_tokens=200)

print("next_question_rag ready")

# %%
# Cell 4 - load or create chat_history (required by extraction)
# Option A: load previously saved chat report
report_path = os.path.join("outputs","report.json")
chat_history = None
if os.path.exists(report_path):
    try:
        with open(report_path,'r',encoding='utf-8') as f:
            saved = json.load(f)
            chat_history = saved.get('chat_history') or saved.get('transcript')
            print('Loaded chat_history from outputs/report.json')
    except Exception as e:
        print('Failed to load saved report:', e)

# Option B: if not available, create a small sample transcript
if not chat_history:
    chat_history = (
        "Patient: I have had chest pain and mild breathlessness since morning.\n"
        "Bot: When did the pain start and does it radiate anywhere?\n"
        "Patient: The pain is sharp and increases on deep breathing. I also feel dizzy.\n"
    )
    print('Using sample chat_history')

print('chat_history length:', len(chat_history))

# %%
# Cell 5 - extraction: ask LLM to return ONLY JSON and robustly parse
import re

schema_instr = (
    'Output ONLY a single valid JSON object with keys:\n'
    '- symptoms: list of strings\n'
    '- duration: string\n'
    '- severity: string (mild/moderate/severe)\n'
    '- medications: list\n'
    '- allergies: list\n'
    '- urgency: string (low/medium/high)\n'
    'Do not output any extra text.'
)

prompt = f"{schema_instr}\n\nConversation:\n{chat_history}\n\nNow output ONLY the JSON."
raw = call_ollama(prompt, max_tokens=600)
print('Raw model output (preview):', raw[:600])

# robust JSON extraction
m = re.search(r'(\{.*\})', raw, re.S)
structured = {}
if m:
    try:
        structured = json.loads(m.group(1))
        print('Structured parsed successfully')
    except Exception as e:
        print('JSON parse failed:', e)
else:
    print('No JSON object found in model output')

print('Structured keys:', list(structured.keys()))

# %%
# Cell 6 - triage and save result
def triage_report(s):
    if not s: return {'specialist':'General Physician','urgency':'low'}
    symptoms = ' '.join(s.get('symptoms',[])).lower()
    if any(w in symptoms for w in ['chest','breath','shortness','palpit']):
        return {'specialist':'Cardiology/Emergency','urgency':'high'}
    if any(w in symptoms for w in ['rash','itch','lesion']):
        return {'specialist':'Dermatology','urgency':'medium'}
    if any(w in symptoms for w in ['headache','dizzy','seizure']):
        return {'specialist':'Neurology','urgency':'medium'}
    return {'specialist':'General Physician','urgency':'low'}

triage = triage_report(structured)
print('Triage result:', triage)

# save
os.makedirs('outputs', exist_ok=True)
out = {'chat_history': chat_history, 'structured': structured, 'triage': triage}
with open('outputs/resume_report.json','w',encoding='utf-8') as f:
    json.dump(out,f,indent=2)
print('Saved outputs/resume_report.json')
